In [1]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from collections import defaultdict
import math

In [66]:
df = pd.read_csv('data/salaries.test.csv')

In [69]:
df['Pos.'].unique()

array(['QB', 'LB', 'WR', 'DB', 'DE', 'C', 'TE', 'T', 'DT', 'G', 'FB', 'RB',
       'LS', 'P', 'K'], dtype=object)

In [70]:
salaries.year.unique()

array([2011, 2012, 2013])

In [111]:
def featureExtractor(filename, features):
    salaries = pd.read_csv(filename)
    teams = salaries.team.unique()
    years = salaries.year.unique()
    allFeatures = []
    for year in years:
        for team in teams:
            result = []
            sliced = salaries[((salaries['team'] == team) & (salaries['year'] == year))]
            teamFeatures = defaultdict(lambda: [])
            for index, row in sliced.iterrows():                
                pos = row['Pos.']
                cap = row['Cap %']
                teamFeatures[pos].append(cap)
            for (position, count) in features:
                posList = sorted(teamFeatures[position], reverse=True)
                for count in range(count):
                    if len(posList) == 0:
                        result.append(0)
                    else:
                        result.append(posList[0])
                        posList = posList[1:]
            allFeatures.append((team, year, result))
    return allFeatures

In [125]:
features = [('QB', 1), ('RB', 1), ('DB', 4), ('LB', 3), ('C', 1), ('DE', 2), ('DT', 2), ('G', 2), ('TE', 2), ('WR', 3), ('T', 2)]
teamFeatures = featureExtractor('data/salaries.train.csv', features)
# for (team, year, vec) in teamFeatures:
#     print team, year, vec

In [126]:
def distance(vec1, vec2):
    return np.linalg.norm(np.array(vec1)-np.array(vec2))

In [131]:
def kNN(featureVec, teamFeatures, k):
    kTop = [float('inf')] * k
    kTeams = [('', 0)] * k
    for (team, year, vec) in teamFeatures:
        dist = distance(vec, featureVec)
        if dist < max(kTop):
            minIndex = kTop.index(max(kTop))
            kTop[minIndex] = dist
            kTeams[minIndex] = (team, year, dist)
    return kTeams

In [133]:
print kNN([1.95, 2.39, 10.16, 3.22, 0.75, 0.55, 14.43, 5.68, 3.53, 0.56, 1.9, 0.8, 2.54, 1.77, 4.54, 0.8, 0.49, 0.44, 2.0, 1.13, 0.54, 4.51, 0.37], teamFeatures, 10)

[('pittsburgh-steelers', 2012, 11.365214472239403), ('kansas-city-chiefs', 2013, 10.250731681202078), ('baltimore-ravens', 2011, 10.996344847266295), ('indianapolis-colts', 2012, 8.5193309596470073), ('denver-broncos', 2011, 0.0), ('baltimore-ravens', 2012, 11.894465099364494), ('baltimore-ravens', 2013, 11.906850969084983), ('indianapolis-colts', 2013, 11.079115488160596), ('new-york-jets', 2012, 10.730745547258122), ('san-francisco-49ers', 2012, 11.587186888973527)]
